## Shazam to Youtube Playlist

In [ ]:
import pandas as pd
from pytube import Search, YouTube
import gradio as gr
from gradio.utils import NamedString

In [ ]:
shazamlibrary_df = pd.read_csv('shazamlibrary.csv', header=1)
shazamlibrary_df

In [ ]:
shazamlibrary_df = shazamlibrary_df.drop_duplicates(subset=['TrackKey'])[['Title', 'Artist']]
shazamlibrary_df

In [ ]:
def get_youtube_song(title: str, artist: str) -> YouTube | None:
    search_result = Search(f'{title} by {artist}')
    return search_result.results[0] if search_result.results else None

In [ ]:
search_title, search_artist = shazamlibrary_df.loc[10, ['Title', 'Artist']].values
youtube: YouTube = get_youtube_song(search_title, search_artist)
youtube.watch_url

In [31]:
def embed_html(playlist: pd.DataFrame):
    search_title, search_artist = playlist.loc[9, ['Title', 'Artist']].values
    youtube: YouTube = get_youtube_song(search_title, search_artist)
    
    iframe = f'''
        <iframe 
            width="100%" 
            height="100%" 
            style="height:100%; width:100%; min-height: 50vh;"
            src="{youtube.embed_url}?autoplay=1"
            frameborder="0" 
            allow="accelerometer; 
            autoplay; 
            clipboard-write; 
            encrypted-media; 
            gyroscope; 
            picture-in-picture" 
            allowfullscreen>
        </iframe>'''
    return gr.HTML(iframe, label="Youtube Player", visible=True)

def upload_file(file: NamedString):
    shazamlibrary_df = pd.read_csv(file, header=1)
    shazamlibrary_df = shazamlibrary_df.drop_duplicates(subset=['TrackKey'])[['Title', 'Artist']]
    playlist = gr.DataFrame(
        shazamlibrary_df, 
        label="Playlist", 
        visible=True,
        column_widths = ['60%', '40%'],
        col_count=(2, 'fixed'),
        row_count=(len(shazamlibrary_df), 'fixed'))
    player = embed_html(shazamlibrary_df)
    return (playlist, player)

with gr.Blocks() as app:
    gr.Markdown("<h1><center>Play your Shazam Playlist from Youtube</center></h1>")
    gr.Markdown("<center>Download the CSV of your playlist from <https://www.shazam.com/myshazam>. </center>")
    gr.Markdown("<center>Upload your Shazam Playlist CSV file.</center>")
    gr.HTML('<hr/>')
    csv = gr.UploadButton(
        label='Upload Shazam Playlist CSV',
        file_count="single",
        file_types = ['.csv'])
    with gr.Row():
        with gr.Column(2):
            player = gr.HTML(label="Youtube Player", visible=False)
        with gr.Column(1):
            playlist = gr.DataFrame(label="Playlist", visible=False)
    csv.upload(upload_file, csv, [playlist, player])

app.launch(debug=True)

Keyboard interruption in main thread... closing server.
